In [ ]:
import numpy as np
import pandas as pd
df = pd.read_csv("C:/Users/ordonez/Documents/UCASAL/notebooks/base_datos_2008.csv", nrows = 1000000)

In [ ]:
display(df)

In [ ]:
#borrar los nulos
df.dropna(inplace=True, subset=["ArrDelay","DepDelay"])

In [ ]:
df.dropna(inplace=True, subset=["DepTime"])

In [ ]:
np.corrcoef([df["ArrDelay"],df["DepDelay"]])

In [ ]:
np.corrcoef([df["ArrDelay"],df["DepDelay"],df["DepTime"]])

In [ ]:
df.drop(inplace=True, columns = ["Year","Cancelled","Diverted"])

In [ ]:
df.drop(inplace=True, columns = ["Month"])

In [ ]:
df.corr()

In [ ]:
corr = round(df.corr(),3)

### Para crear la matriz de coorrelación es necesario instalar matplotlib

In [ ]:
corr.style.background_gradient()

# Test de chi-Cuadrado
Criterio para encontrar relación entre variables cualitativas
- Se utiliza para comparar proporciones
- Se basa sobre el análisis de variables cualtitativas
- Nos va a decir si una tasa es diferente a otra y si esta diferencia es estadisticamente significativa
- Para calcularla nos auxiliamos de las **tablas de contingecia**
$$X^2=\sum_{i=1}^k \frac{(observado_i - esperado_i)^2}{esperado_i}$$

In [ ]:
import pandas as pd
import numpy as np
import time
start = time.time()
df = pd.read_csv("C:/Users/ordonez/Documents/UCASAL/notebooks/base_datos_2008.csv")
end = time.time()
print(f"Se tardó {round(end-start,2)} segundos en cargar {len(df)} registros.")

In [ ]:
np.random.seed(0)
df = df[df["Origin"].isin(["HOU","ATL","IND"])]
df = df.sample(frac=1)
df = df[0:10000]

In [ ]:
df["BigDelay"] = df["ArrDelay"] > 30
observados = pd.crosstab(index=df['BigDelay'],columns=df['Origin'], margins=True)

In [ ]:
observados

In [ ]:
from scipy.stats import chi2_contingency

In [ ]:
test = chi2_contingency(observados)

In [ ]:
test

In [ ]:
esperados = pd.DataFrame(test[3])

In [ ]:
esperados

In [ ]:
esperados_rel = round(esperados.apply(lambda r: r/len(df) * 100,axis=1),2)

In [ ]:
observados_rel = round(observados.apply(lambda r: r/len(df) * 100,axis=1),2)

In [ ]:
observados_rel

In [ ]:
esperados_rel

In [ ]:
test[1]

## Resumen de Test de Hipótesis
* Si el p-valor < 0.05, hay diferencias significativas: Hay relación entre variables
* Si el p-valor > 0.05, no hay difrencias significativas: No hay relación entre variables

# Análisis de datos extremos

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv("C:/Users/ordonez/Documents/UCASAL/notebooks/base_datos_2008.csv", nrows = 100000)

In [ ]:
x = df["ArrDelay"].dropna()

In [ ]:
Q1 = np.percentile(x,25)
Q3 = np.percentile(x,75)
rangointer = Q3 - Q1

In [ ]:
umbralsuperior = Q3 + 1.5*rangointer
umbralinferior = Q1 - 1.5*rangointer

In [ ]:
umbralsuperior

In [ ]:
umbralinferior

In [ ]:
np.mean(x > umbralsuperior)

In [ ]:
np.mean(x < umbralinferior)

In [ ]:
from sklearn.covariance import EllipticEnvelope

In [ ]:
outliers = EllipticEnvelope(contamination = .01)

In [ ]:
var_list = ["DepDelay","TaxiIn","TaxiOut","CarrierDelay","WeatherDelay","NASDelay","SecurityDelay","LateAircraftDelay"]

In [ ]:
x = np.array(df.loc[:,var_list].dropna())

In [ ]:
outliers.fit(x)

In [ ]:
pred = outliers.predict(x)

In [ ]:
pred

In [ ]:
elips_outliers = np.where(pred == -1)[0]

In [ ]:
#conseguiremos un listado de los index de los casos que están alejados de los valores centrales de todas las variables estudiadas
elips_outliers

# Principios de las bases de datos relacionales
## Transformar una dataframe en una base de datos relacional

In [ ]:
import pandas as pd
data = [(1,"Joan","Gasull",25,1,"Libreta",1.2,.4,0.8,3,"03-02-2018"),
       (1,"Joan","Gasull",25,2,"Bolígrafo",0.4,0.15,0.25,1,"03-02-2018"),
       (1,"Joan","Gasull",25,1,"Libreta",1.2,.4,0.8,2,"15-02-2018"),
       (2,"Joan","López",33,2,"Bolígrafo",0.4,0.15,0.25,4,"01-02-2018"),
       (2,"Joan","López",33,1,"Libreta",1.2,.4,0.8,10,"05-03-2018"),
       (3,"Maria","Garcia",40,1,"Libreta",1.2,.4,0.8,20,"13-04-2018"),
       (3,"Maria","Garcia",40,2,"Bolígrafo",0.4,0.15,0.25,1,"09-02-2018"),
       (3,"Maria","Garcia",40,2,"Bolígrafo",0.4,0.15,0.25,3,"03-04-2018")]
labels = ["Comprador_id","Nombre","Apellido","Edad","Producto_id","Producto",
         "Precio","Coste","Margen","Cantidad","Fecha"]

In [ ]:
type(data)

In [ ]:
type(data[0])

In [ ]:
df = pd.DataFrame.from_records(data,columns = labels)

In [ ]:
df

In [ ]:
compradores = df.drop_duplicates(subset = "Comprador_id", keep = "first")
compradores = compradores[["Comprador_id","Nombre","Apellido","Edad"]]
compradores

In [ ]:
productos = df.drop_duplicates(subset="Producto_id", keep="first")
productos = productos[["Producto_id","Producto","Precio","Margen"]]
productos

In [ ]:
compras = df[["Comprador_id","Producto_id","Fecha","Cantidad"]]
compras

## Joins. Trabajar con bases de datos relacionales

In [ ]:
import pandas as pd
consumidores = [("A","Móvil"),("B","Móvil"),("A","Portátil"),("A","Tablet"),
               ("B","Tablet"),("C","Portátil"),("D","Smartwatch"),("E","Consola")]
con_labels = ["Consumidor","Producto"]
con_df = pd.DataFrame.from_records(consumidores, columns = con_labels)
productores = [("a","Móvil"),("a","Smartwatch"),("a","Tablet"),("b","Portátil"),
               ("c","Sobremesa"),("c","Portátil")]
prod_labels = ["Productor","Producto"]
prod_df = pd.DataFrame.from_records(productores, columns=prod_labels)

In [ ]:
con_df

In [ ]:
prod_df

In [ ]:
pd.merge(con_df,prod_df,on="Producto",how="outer")

In [ ]:
pd.merge(con_df,prod_df,on="Producto",how="inner")

In [ ]:
pd.merge(con_df,prod_df,on="Producto",how="right")

## Paralelizar loops en Python

In [ ]:
import pandas as pd
import numpy as np
from joblib import Parallel, delayed

df = pd.read_csv("C:/Users/ordonez/Documents/UCASAL/notebooks/base_datos_2008.csv", nrows = 100000)